In [7]:
import decimal
ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third
one_third == +one_third

True

In [8]:
one_third == +one_third

False

In [9]:
+one_third

Decimal('0.3333333333333333333333333333')

In [11]:
from collections import Counter
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [12]:
ct['r'] = -3
ct['d'] = 0

In [13]:
ct

Counter({'a': 5, 'b': 2, 'r': -3, 'c': 1, 'd': 0})

In [14]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

In [40]:
import operator
import operator
import itertools
import numbers
from array import array
import reprlib

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+bytes(self.comonents))
    
#     def __eq__(self, other):
# #         return tuple(self) == tuple(other)

#         if len(self) != len(other):
#             return Flase
# #         for a, b in zip(self, other):
# #             if a != b:
# #                 return False
# #         return True
# #       使用all
#         return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
    def __getattr__(self, name):
        cls = type(self)
        
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_name:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def __hash__(self):
#         hashes = (hash(x) for x in self._components)
#         return functools.reduce(operator.xor, hashes, 0)
#       使用map
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes)
    
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) -1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def  __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
        
    def __rmul__(self, scalar):
        return self * scalar
    
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented
        
    def __rmatmul__(self, other):
        return self @ other
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
        
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
        
    def __radd__(self, other):
        return self + other
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(cotets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [26]:
v1 = Vector([1.0, 2.0, 3.0])
14 * v1

Vector([14.0, 28.0, 42.0])

In [27]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [28]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

In [29]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz == 38.0

True

In [30]:
[10, 20, 30] @ vz

380.0

In [31]:
va @ 3

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

In [32]:
va

Vector([1.0, 2.0, 3.0])

In [33]:
vb = Vector(range(1, 4))
va == vb

True

In [34]:
vc = Vector([1, 2])
t3 = (1, 2, 3)
va == t3

False

In [35]:
va != vb

False

In [36]:
vc != (1, 2, 3)

True

In [46]:
v1 = Vector([1, 2, 3])
v1_alias = v1
print(v1)
v1 += Vector([4, 5, 6])
print(v1)
print(id(v1))
print(v1_alias)
v1 *= 11
print(v1)
print(id(v1))

(1.0, 2.0, 3.0)
(5.0, 7.0, 9.0)
2183433927424
(1.0, 2.0, 3.0)
(55.0, 77.0, 99.0)
2183433927872


In [48]:
import random
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """"""
        
    @abc.abstractmethod
    def pick(self):
        """"""
    def loaded(self):
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))
    
class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [49]:
import itertools

class AddableBingoCage(BingoCage):
    
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented
        
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self

In [50]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [51]:
globe.pick() in vowels

True

In [52]:
len(globe.inspect())

4

In [53]:
globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
len(globe3.inspect())

7

In [54]:
void = globe + [10, 20]

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [55]:
globe_orig = globe
len(globe.inspect())

4

In [56]:
globe += globe2
len(globe.inspect())

7

In [57]:
globe +=['M', 'N']
len(globe.inspect())

9

In [58]:
globe is globe_orig

True

In [59]:
globe += 1

TypeError: right operand in += must be 'AddableBingoCage' or an iterable

In [60]:
globe

In [61]:
globe.inspect()

('A', 'I', 'M', 'N', 'O', 'U', 'X', 'Y', 'Z')

In [65]:
globe += [10, 20]

In [66]:
globe.inspect()

TypeError: '<' not supported between instances of 'str' and 'int'

In [78]:
from numbers import Real
n = numbers
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))
    
print_mro(numbers)

AttributeError: module 'numbers' has no attribute '__mro__'

## decimal.Macimal设置算术运算上下文的精度
- 精度变化可能导致x不等于+x
```python
import decimal
ctx = decimal.getcontext()
# 默认是28位精度
ctx.prec = 40
ont_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third
one_third == +one_third
# 设为28位
# 这里如果是在notebook中，同一行算一个上下文，ctx.prec只对同一上下文起作用
ctx.prec = 28
one_third == +one_third # 此时不相等
+one_third
```
- +Counter仅保留大于零的计数器
```python
ct = Counter('abracadabra')
ct
ct['r'] = -3
ct['d'] = 0
ct
+ct # 只保留大于零的计数
```
## 重载向量加法运算符
```python
···
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
        
    def __radd__(self, other):
        return self + other
···
# 正常情况下是：v1 + (10, 20, 30)
# 而(10, 20, 30) + v1 调用的是tuple的__add__方法，由于没有Vector，会报TypeError
# 左操作数首先调用__add__方法，如果返回NotImplemented表明它不知道如何处理右操作数，那么就会调用__radd__方法
# NotImplemented是特殊的单例值，不是异常
# __radd__可以实现(10, 20, 30) + v1
```
## 重载标量乘法运算符
```python
···
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
        
    def __rmul__(self, scalar):
        return self * scalar
···
v1 = Vector([1.0, 2.0, 3.0])
14 * v1
```
## 改进的Vector.__eq__方法
```python
···
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
···
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb
vc = Vector([1, 2])
t3 = (1, 2, 3)
va == t3
```
## 增量赋值运算符
- 增量赋值不会修改不可变目标，而是新建实例，然后重新绑定
如果类没有实现__iadd__()方法，a += b 与 a = a + b 一样，对不可变类型是预期的行为。
```python
v1 = Vector([1, 2, 3])
v1_alias = v1 # 保留v1副本v1_alias
id(v1)
v1 += Vector([4, 5, 6])
v1 
id(v1) # 此时的v1 id改变了 
v1_alias # 原来的实例没有改变

v1 *= 11
v1
id(v1) # 与增量加法一样
v1_alias
```
## 自增运算符
```python
import random
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """"""
        
    @abc.abstractmethod
    def pick(self):
        """"""
    def loaded(self):
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))
    
class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()
        
import itertools

class AddableBingoCage(BingoCage):
    
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented
        
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self
    
# 加运算符
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe.inspect()
globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
len(globe3.inspect())
void = globe + [10, 20] # AddableBingoCage实例无法与列表相加
# 自加运算符
globe += globe2
globe.inspect()
globe += ['M', 'N']
globe.inspect()
globe is globe_orig
globe += 1 # AddableBingoCage实例不能与非可迭代对象相加
```
## 总结
- decimal.Decimal设置上下文精度
- 重加载__add__方法，NotImplemented单例值，以及__radd__方法的作用
- 使用isinstance判断重载标量乘法运算符的参数检查numbers.Real抽象基类，numbers.Real涵盖类我们所需的全部类型
- __eq__方法中使用isinstance判断类型
- 增量赋值运算符对可变或不可变实例的行为
- 加运算符和自加运算符的差别